In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./activity.csv',sep=',')
df = df.dropna()
df=df.drop(["company_type","field","target"],axis='columns')


#기업명
df["company_name"] = df["company_name"].apply(lambda x: re.sub('[|0-9기]', '', x))      # ‘| 1기’ 와 같은 기수 제외

#관심분야 더미변수화
df = df.join(df["interest"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())                                                             #디자인사진예술영상 열, 콘텐츠열 삭제


#우대역량 더미변수화
df["prefer"] = df["prefer"].str.replace(", 기타", "")                                                      #',기타'가 규칙없이 붙어있는경우가 많아서 삭제
df['prefer'] = df['prefer'].replace('기타', '우대역량_기타')                                                 #'기타'열이 관심분야에도 중복됨으로 교체 
df = df.join(df["prefer"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#모집인원
df["모집 인원"]=df["recruit"].str.extract(r'(\d+)')     #숫자만 남기기
df=df.replace({"모집 인원":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"모집 인원":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"모집 인원":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"모집 인원":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"모집 인원":'int'})                   
student_bins=[0,1,21,101,50000]                        #범주화  0:인원미정/1-20(명):소/21-100(명):중/101-50000(명):대
student_scale=["인원미정","소","중","대"]               
df = pd.concat([df, pd.cut(df["모집 인원"], bins=student_bins, right=False, labels=student_scale).rename("모집인원규모")], axis=1) #분포범주화



#열삭제
df=df.drop(["activity_name","app_period","area","activity_period"],axis='columns')

#열위치조정
df = df.reindex(columns=["idx","company_name","interests","경영컨설팅마케팅","경제금융","과학공학기술IT",
                         "교육","문화역사","뷰티미용화장품","사회공헌교류","언론미디어","여행호텔항공",
                         "요리식품","의료보건","정치사회법률","창업자기계발","행사페스티벌","환경에너지",
                         "기타체육헬스","디자인사진예술영상콘텐츠","prefer","리더십","멘토링/수업 진행 경험",
                         "모두 지원 가능","사진/영상/디자인 툴","외국어","우대역량_기타","취재/인터뷰/기사 경험",
                         "컴퓨터 활용능력","콘텐츠 기획/제작 경험","타 대외활동 경험","파워블로거/SNS",
                         "행사 기획 경험","휴학 중/시간 투자 가능","모집 인원","모집인원규모"
                         ])

#데이터저장
df=df.to_csv("activity_process.csv")
